# অ্যাটেনশন মেকানিজম এবং ট্রান্সফর্মার

রেকারেন্ট নেটওয়ার্কগুলোর একটি বড় সীমাবদ্ধতা হলো, একটি সিকোয়েন্সের সব শব্দ ফলাফলের উপর সমান প্রভাব ফেলে। এর ফলে সিকোয়েন্স-টু-সিকোয়েন্স কাজের জন্য স্ট্যান্ডার্ড LSTM এনকোডার-ডিকোডার মডেলগুলোতে অপ্টিমাল পারফরম্যান্স হয় না, যেমন নেমড এন্টিটি রিকগনিশন এবং মেশিন ট্রান্সলেশন। বাস্তবে, ইনপুট সিকোয়েন্সের নির্দিষ্ট কিছু শব্দ সিকোয়েন্সিয়াল আউটপুটে অন্য শব্দগুলোর চেয়ে বেশি প্রভাব ফেলে।

ধরা যাক একটি সিকোয়েন্স-টু-সিকোয়েন্স মডেল, যেমন মেশিন ট্রান্সলেশন। এটি দুটি রেকারেন্ট নেটওয়ার্ক দ্বারা বাস্তবায়িত হয়, যেখানে একটি নেটওয়ার্ক (**encoder**) ইনপুট সিকোয়েন্সকে হিডেন স্টেটে সংকুচিত করে এবং অন্যটি, **decoder**, এই হিডেন স্টেটকে অনুবাদিত ফলাফলে রূপান্তরিত করে। এই পদ্ধতির সমস্যা হলো, নেটওয়ার্কের চূড়ান্ত স্টেটের জন্য বাক্যের শুরুটা মনে রাখা কঠিন হয়ে যায়, যার ফলে দীর্ঘ বাক্যের ক্ষেত্রে মডেলের গুণগত মান খারাপ হয়।

**অ্যাটেনশন মেকানিজম** RNN-এর প্রতিটি আউটপুট প্রেডিকশনে প্রতিটি ইনপুট ভেক্টরের প্রাসঙ্গিক প্রভাবকে ওজন দেওয়ার একটি উপায় প্রদান করে। এটি বাস্তবায়িত হয় ইনপুট RNN-এর মধ্যবর্তী স্টেট এবং আউটপুট RNN-এর মধ্যে শর্টকাট তৈরি করে। এই পদ্ধতিতে, যখন আউটপুট প্রতীক $y_t$ তৈরি করা হয়, তখন আমরা বিভিন্ন ওজন সহগ $\alpha_{t,i}$ সহ সমস্ত ইনপুট হিডেন স্টেট $h_i$ বিবেচনায় নেব। 

![এনকোডার/ডিকোডার মডেল একটি অ্যাডিটিভ অ্যাটেনশন লেয়ার সহ](../../../../../translated_images/bn/encoder-decoder-attention.7a726296894fb567.webp)
*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) থেকে উদ্ধৃত, [এই ব্লগ পোস্ট](https://lilianweng.github.io/lil-log/2018/06/24/attention-attention.html) থেকে নেওয়া এনকোডার-ডিকোডার মডেল অ্যাডিটিভ অ্যাটেনশন মেকানিজম সহ*

অ্যাটেনশন ম্যাট্রিক্স $\{\alpha_{i,j}\}$ একটি নির্দিষ্ট আউটপুট সিকোয়েন্সের একটি শব্দ তৈরিতে কোন ইনপুট শব্দগুলো কতটা ভূমিকা রাখছে তা উপস্থাপন করে। নিচে এমন একটি ম্যাট্রিক্সের উদাহরণ দেওয়া হলো:

![RNNsearch-50 দ্বারা পাওয়া একটি নমুনা অ্যালাইনমেন্ট, Bahdanau - arviz.org থেকে নেওয়া](../../../../../translated_images/bn/bahdanau-fig3.09ba2d37f202a6af.webp)

*[Bahdanau et al., 2015](https://arxiv.org/pdf/1409.0473.pdf) থেকে নেওয়া (Fig.3)*

অ্যাটেনশন মেকানিজম বর্তমান বা প্রায় বর্তমান প্রাকৃতিক ভাষা প্রক্রিয়াকরণের (NLP) স্টেট অফ দ্য আর্টের জন্য দায়ী। তবে অ্যাটেনশন যোগ করার ফলে মডেলের প্যারামিটারের সংখ্যা উল্লেখযোগ্যভাবে বৃদ্ধি পায়, যা RNN-এর স্কেলিং সমস্যার দিকে নিয়ে যায়। RNN-এর স্কেলিংয়ের একটি মূল সীমাবদ্ধতা হলো, মডেলের রেকারেন্ট প্রকৃতি প্রশিক্ষণকে ব্যাচ এবং প্যারালালাইজ করা কঠিন করে তোলে। RNN-এ একটি সিকোয়েন্সের প্রতিটি উপাদানকে ক্রমানুসারে প্রক্রিয়া করতে হয়, যার ফলে এটি সহজে প্যারালালাইজ করা যায় না।

অ্যাটেনশন মেকানিজমের গ্রহণযোগ্যতা এবং এই সীমাবদ্ধতার সংমিশ্রণে, আজকের স্টেট অফ দ্য আর্ট ট্রান্সফর্মার মডেলগুলোর সৃষ্টি হয়েছে, যা আমরা BERT থেকে OpenGPT3 পর্যন্ত ব্যবহার করি।

## ট্রান্সফর্মার মডেল

প্রতিটি পূর্ববর্তী প্রেডিকশনের প্রসঙ্গকে পরবর্তী মূল্যায়ন ধাপে ফরোয়ার্ড করার পরিবর্তে, **ট্রান্সফর্মার মডেলগুলো** **পজিশনাল এনকোডিং** এবং **অ্যাটেনশন** ব্যবহার করে একটি প্রদত্ত টেক্সট উইন্ডোর মধ্যে একটি ইনপুটের প্রসঙ্গ ধারণ করে। নিচের চিত্রটি দেখায় কীভাবে পজিশনাল এনকোডিং এবং অ্যাটেনশন একটি প্রদত্ত উইন্ডোর মধ্যে প্রসঙ্গ ধারণ করতে পারে।

![ট্রান্সফর্মার মডেলে মূল্যায়ন কীভাবে সম্পন্ন হয় তা দেখানো একটি অ্যানিমেটেড GIF।](../../../../../lessons/5-NLP/18-Transformers/images/transformer-animated-explanation.gif) 

যেহেতু প্রতিটি ইনপুট পজিশন স্বাধীনভাবে প্রতিটি আউটপুট পজিশনে ম্যাপ করা হয়, ট্রান্সফর্মারগুলো RNN-এর চেয়ে ভালোভাবে প্যারালালাইজ করতে পারে, যা অনেক বড় এবং আরও এক্সপ্রেসিভ ভাষার মডেল সক্ষম করে। প্রতিটি অ্যাটেনশন হেড বিভিন্ন শব্দের মধ্যে সম্পর্ক শিখতে ব্যবহার করা যেতে পারে, যা ডাউনস্ট্রিম NLP কাজগুলো উন্নত করে।

## সহজ ট্রান্সফর্মার মডেল তৈরি

Keras-এ বিল্ট-ইন ট্রান্সফর্মার লেয়ার নেই, তবে আমরা নিজেরাই এটি তৈরি করতে পারি। আগের মতো, আমরা AG News ডেটাসেটের টেক্সট ক্লাসিফিকেশনের উপর ফোকাস করব, তবে এটি উল্লেখযোগ্য যে ট্রান্সফর্মার মডেলগুলো আরও কঠিন NLP কাজগুলোতে সেরা ফলাফল দেখায়।


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

কেরাস-এ নতুন লেয়ারগুলি `Layer` ক্লাসকে সাবক্লাস করা উচিত এবং `call` মেথডটি ইমপ্লিমেন্ট করা উচিত। চলুন **Positional Embedding** লেয়ার দিয়ে শুরু করি। আমরা [কেরাসের অফিসিয়াল ডকুমেন্টেশন থেকে কিছু কোড](https://keras.io/examples/nlp/text_classification_with_transformer/) ব্যবহার করব। আমরা ধরে নেব যে আমরা সমস্ত ইনপুট সিকোয়েন্সকে `maxlen` দৈর্ঘ্যে প্যাড করি।


In [2]:
class TokenAndPositionEmbedding(keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = keras.layers.Embedding(input_dim=maxlen, output_dim=embed_dim)
        self.maxlen = maxlen

    def call(self, x):
        maxlen = self.maxlen
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x+positions

এই স্তরটি দুটি `Embedding` স্তর নিয়ে গঠিত: একটি টোকেন এমবেড করার জন্য (যেমনটি আমরা আগে আলোচনা করেছি) এবং আরেকটি টোকেন পজিশনের জন্য। টোকেন পজিশনগুলো `tf.range` ব্যবহার করে 0 থেকে `maxlen` পর্যন্ত প্রাকৃতিক সংখ্যার একটি ক্রম হিসেবে তৈরি করা হয় এবং তারপর এমবেডিং স্তরের মাধ্যমে পাঠানো হয়। দুটি প্রাপ্ত এমবেডিং ভেক্টর যোগ করা হয়, যার ফলে ইনপুটের একটি পজিশনালি এমবেড করা উপস্থাপনা তৈরি হয় যার আকার `maxlen`$\times$`embed_dim`।

এখন, চলুন ট্রান্সফরমার ব্লকটি বাস্তবায়ন করি। এটি পূর্বে সংজ্ঞায়িত এমবেডিং স্তরের আউটপুট গ্রহণ করবে:


In [3]:
class TransformerBlock(keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim, name='attn')
        self.ffn = keras.Sequential(
            [keras.layers.Dense(ff_dim, activation="relu"), keras.layers.Dense(embed_dim),]
        )
        self.layernorm1 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = keras.layers.Dropout(rate)
        self.dropout2 = keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

এখন আমরা সম্পূর্ণ ট্রান্সফর্মার মডেল সংজ্ঞায়িত করতে প্রস্তুত:


In [4]:
embed_dim = 32  # Embedding size for each token
num_heads = 2  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
maxlen = 256
vocab_size = 20000

model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_sequence_length=maxlen, input_shape=(1,)),
    TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim),
    TransformerBlock(embed_dim, num_heads, ff_dim),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(20, activation="relu"),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(4, activation="softmax")
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 256)               0         
_________________________________________________________________
token_and_position_embedding (None, 256, 32)           648192    
_________________________________________________________________
transformer_block (Transform (None, 256, 32)           10656     
_________________________________________________________________
global_average_pooling1d (Gl (None, 32)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 20)                660       
_________________________________________________________________
dropout_3 (Dropout)          (None, 20)               

In [5]:
print('Training tokenizer')
model.layers[0].adapt(ds_train.map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Training tokenizer
938/938 [==============================] - 45s 39ms/step - loss: 0.4978 - acc: 0.8068 - val_loss: 0.2808 - val_acc: 0.9124


## BERT ট্রান্সফর্মার মডেল

**BERT** (Bidirectional Encoder Representations from Transformers) একটি অত্যন্ত বড় মাল্টি-লেয়ার ট্রান্সফর্মার নেটওয়ার্ক, যেখানে *BERT-base* এর জন্য ১২টি স্তর এবং *BERT-large* এর জন্য ২৪টি স্তর রয়েছে। এই মডেলটি প্রথমে বিশাল পরিমাণ টেক্সট ডেটা (উইকিপিডিয়া + বই) ব্যবহার করে অ-পর্যবেক্ষণমূলক প্রশিক্ষণের মাধ্যমে (একটি বাক্যে মাস্ক করা শব্দ অনুমান করা) প্রি-ট্রেইন করা হয়। প্রি-ট্রেইনিংয়ের সময় মডেলটি উল্লেখযোগ্য ভাষাগত বোঝাপড়া অর্জন করে, যা পরে অন্যান্য ডেটাসেটের সাথে ফাইন টিউনিংয়ের মাধ্যমে ব্যবহার করা যায়। এই প্রক্রিয়াকে **ট্রান্সফার লার্নিং** বলা হয়।

![http://jalammar.github.io/illustrated-bert/ থেকে ছবি](../../../../../translated_images/bn/jalammarBERT-language-modeling-masked-lm.34f113ea5fec4362.webp)

ট্রান্সফর্মার আর্কিটেকচারের অনেক বৈচিত্র্য রয়েছে, যেমন BERT, DistilBERT, BigBird, OpenGPT3 এবং আরও অনেক কিছু, যেগুলো ফাইন টিউন করা যেতে পারে।

চলুন দেখি কীভাবে আমরা প্রি-ট্রেইন করা BERT মডেল ব্যবহার করে আমাদের প্রচলিত সিকোয়েন্স ক্লাসিফিকেশন সমস্যার সমাধান করতে পারি। আমরা [অফিশিয়াল ডকুমেন্টেশন](https://www.tensorflow.org/text/tutorials/classify_text_with_bert) থেকে ধারণা এবং কিছু কোড ধার করব।

প্রি-ট্রেইন করা মডেল লোড করতে আমরা **Tensorflow hub** ব্যবহার করব। প্রথমে, চলুন BERT-স্পেসিফিক ভেক্টরাইজার লোড করি:


In [1]:
import tensorflow_text 
import tensorflow_hub as hub
vectorizer = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')

ModuleNotFoundError: No module named 'tensorflow_text'

In [7]:
vectorizer(['I love transformers'])

{'input_type_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
       dtype=int32)>,
 'input_word_ids': <tf.Tensor: shape=(1, 128), dtype=int32, numpy=
 array([[  101,  1045,  2293, 19081,   102,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

এটি গুরুত্বপূর্ণ যে আপনি একই ভেক্টরাইজার ব্যবহার করেন যেটি মূল নেটওয়ার্ক প্রশিক্ষণের জন্য ব্যবহৃত হয়েছিল। এছাড়াও, BERT ভেক্টরাইজার তিনটি উপাদান প্রদান করে:
* `input_word_ids`, যা ইনপুট বাক্যের জন্য টোকেন নম্বরগুলোর একটি ক্রম
* `input_mask`, যা দেখায় ক্রমের কোন অংশটি প্রকৃত ইনপুট এবং কোন অংশটি প্যাডিং। এটি `Masking` লেয়ার দ্বারা উৎপন্ন মাস্কের মতো
* `input_type_ids` ভাষা মডেলিং কাজের জন্য ব্যবহৃত হয় এবং একটি ক্রমে দুটি ইনপুট বাক্য নির্দিষ্ট করার অনুমতি দেয়।

এরপর, আমরা BERT ফিচার এক্সট্রাক্টর ইনস্ট্যান্সিয়েট করতে পারি:


In [8]:
bert = hub.KerasLayer('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-128_A-2/1')

In [9]:
z = bert(vectorizer(['I love transformers']))
for i,x in z.items():
    print(f"{i} -> { len(x) if isinstance(x, list) else x.shape }")

pooled_output -> (1, 128)
encoder_outputs -> 4
sequence_output -> (1, 128, 128)
default -> (1, 128)


তাহলে, BERT স্তর বেশ কিছু দরকারী ফলাফল প্রদান করে:
* `pooled_output` হলো সিকোয়েন্সের সমস্ত টোকেনের গড় হিসাব। আপনি এটিকে পুরো নেটওয়ার্কের একটি বুদ্ধিমান অর্থবোধক এমবেডিং হিসেবে দেখতে পারেন। এটি আমাদের আগের মডেলের `GlobalAveragePooling1D` স্তরের আউটপুটের সমতুল্য।
* `sequence_output` হলো শেষ ট্রান্সফর্মার স্তরের আউটপুট (উপরের মডেলে `TransformerBlock` এর আউটপুটের সাথে মিলে যায়)।
* `encoder_outputs` হলো সমস্ত ট্রান্সফর্মার স্তরের আউটপুট। যেহেতু আমরা ৪-স্তরের BERT মডেল লোড করেছি (যা আপনি সম্ভবত নাম থেকে অনুমান করতে পারেন, যেখানে `4_H` রয়েছে), এটি ৪টি টেনসর প্রদান করে। শেষটি `sequence_output` এর সমান।

এখন আমরা এন্ড-টু-এন্ড ক্লাসিফিকেশন মডেল সংজ্ঞায়িত করব। আমরা *ফাংশনাল মডেল সংজ্ঞা* ব্যবহার করব, যেখানে আমরা মডেলের ইনপুট সংজ্ঞায়িত করব এবং তারপর এর আউটপুট গণনা করার জন্য একাধিক এক্সপ্রেশন প্রদান করব। আমরা BERT মডেলের ওজনগুলোকে ট্রেনিং-অযোগ্য করব এবং শুধুমাত্র চূড়ান্ত ক্লাসিফায়ারটি ট্রেন করব:


In [10]:
inp = keras.Input(shape=(),dtype=tf.string)
x = vectorizer(inp)
x = bert(x)
x = keras.layers.Dropout(0.1)(x['pooled_output'])
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
bert.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer='adam')
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

938/938 [==============================] - 528s 559ms/step - loss: 0.8056 - acc: 0.6983 - val_loss: 0.5953 - val_acc: 0.7888


যদিও ট্রেন করার মতো প্যারামিটার খুব কম, প্রক্রিয়াটি বেশ ধীর, কারণ BERT ফিচার এক্সট্রাক্টর কম্পিউটেশনালভাবে ভারী। মনে হচ্ছে আমরা যুক্তিসঙ্গত সঠিকতা অর্জন করতে পারিনি, হয় প্রশিক্ষণের অভাবে, অথবা মডেলের প্যারামিটারগুলির অভাবে।

চলুন BERT এর ওজন আনফ্রিজ করে এটিকে ট্রেন করার চেষ্টা করি। এর জন্য খুব ছোট লার্নিং রেট প্রয়োজন, এবং **warmup** সহ আরও সতর্ক প্রশিক্ষণ কৌশল প্রয়োজন, **AdamW** অপ্টিমাইজার ব্যবহার করে। আমরা `tf-models-official` প্যাকেজ ব্যবহার করে অপ্টিমাইজার তৈরি করব:


In [12]:
from official.nlp import optimization 
bert.trainable=True
model.summary()
epochs = 3
opt = optimization.create_optimizer(
    init_lr=3e-5,
    num_train_steps=epochs*len(ds_train),
    num_warmup_steps=0.1*epochs*len(ds_train),
    optimizer_type='adamw')

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'], optimizer=opt)
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128))

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        {'input_type_ids': ( 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_1 (KerasLayer)      {'pooled_output': (N 4782465     keras_layer[0][0]                
                                                                 keras_layer[0][1]                
                                                                 keras_layer[0][2]                
______________________________________________________________________________________________

যেমনটি দেখা যাচ্ছে, প্রশিক্ষণ প্রক্রিয়া বেশ ধীরগতিতে চলছে - তবে আপনি চাইলে কিছু ইপোক (৫-১০) নিয়ে পরীক্ষা করতে পারেন এবং আগের ব্যবহৃত পদ্ধতির সাথে তুলনা করে সেরা ফলাফল পেতে চেষ্টা করতে পারেন।

## Huggingface Transformers লাইব্রেরি

Transformer মডেল ব্যবহারের আরেকটি খুব সাধারণ (এবং কিছুটা সহজ) উপায় হলো [HuggingFace প্যাকেজ](https://github.com/huggingface/), যা বিভিন্ন NLP কাজের জন্য সহজ বিল্ডিং ব্লক সরবরাহ করে। এটি Tensorflow এবং PyTorch উভয়ের জন্যই উপলব্ধ, যা আরেকটি জনপ্রিয় নিউরাল নেটওয়ার্ক ফ্রেমওয়ার্ক।

> **Note**: যদি আপনি Transformers লাইব্রেরি কীভাবে কাজ করে তা দেখতে আগ্রহী না হন - তাহলে আপনি এই নোটবুকের শেষে চলে যেতে পারেন, কারণ এখানে আমরা উপরে যা করেছি তার থেকে মৌলিকভাবে ভিন্ন কিছু দেখতে পাবেন না। আমরা BERT মডেল প্রশিক্ষণের একই ধাপগুলি পুনরায় ব্যবহার করব, তবে একটি ভিন্ন লাইব্রেরি এবং উল্লেখযোগ্যভাবে বড় মডেল নিয়ে। সুতরাং, প্রক্রিয়াটি কিছুটা দীর্ঘ প্রশিক্ষণ জড়িত, তাই আপনি শুধু কোডটি দেখে নিতে চাইতে পারেন।

চলুন দেখি আমাদের সমস্যাটি কীভাবে [Huggingface Transformers](http://huggingface.co) ব্যবহার করে সমাধান করা যায়।


প্রথমে আমাদের যে মডেলটি ব্যবহার করব তা নির্বাচন করতে হবে। কিছু বিল্ট-ইন মডেলের পাশাপাশি, Huggingface-এ একটি [অনলাইন মডেল সংগ্রহশালা](https://huggingface.co/models) রয়েছে, যেখানে আপনি কমিউনিটির তৈরি আরও অনেক প্রি-ট্রেইনড মডেল খুঁজে পেতে পারেন। এই সমস্ত মডেল শুধুমাত্র মডেলের নাম প্রদান করেই লোড এবং ব্যবহার করা যায়। মডেলের জন্য প্রয়োজনীয় সমস্ত বাইনারি ফাইল স্বয়ংক্রিয়ভাবে ডাউনলোড হয়ে যাবে।

কিছু সময়ে আপনাকে নিজের মডেল লোড করতে হতে পারে, সেই ক্ষেত্রে আপনি সেই ডিরেক্টরিটি নির্দিষ্ট করতে পারেন যেখানে সমস্ত প্রাসঙ্গিক ফাইল রয়েছে, যেমন টোকেনাইজারের জন্য প্যারামিটার, `config.json` ফাইলের মডেল প্যারামিটার, বাইনারি ওজন ইত্যাদি।

মডেলের নাম থেকে আমরা মডেল এবং টোকেনাইজার উভয়ই ইনস্ট্যানশিয়েট করতে পারি। চলুন টোকেনাইজার দিয়ে শুরু করি:


In [2]:
import transformers

# To load the model from Internet repository using model name. 
# Use this if you are running from your own copy of the notebooks
bert_model = 'bert-base-uncased' 

# To load the model from the directory on disk. Use this for Microsoft Learn module, because we have
# prepared all required files for you.
#bert_model = './bert'

tokenizer = transformers.BertTokenizer.from_pretrained(bert_model)

MAX_SEQ_LEN = 128
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

`tokenizer` অবজেক্টে `encode` ফাংশন রয়েছে যা সরাসরি টেক্সট এনকোড করতে ব্যবহার করা যেতে পারে:


In [3]:
tokenizer.encode('Tensorflow is a great framework for NLP')

[101, 23435, 12314, 2003, 1037, 2307, 7705, 2005, 17953, 2361, 102]

আমরা টোকেনাইজার ব্যবহার করে একটি সিকোয়েন্স এমনভাবে এনকোড করতে পারি যা মডেলে পাঠানোর জন্য উপযুক্ত, যেমন `token_ids`, `input_mask` ফিল্ড ইত্যাদি অন্তর্ভুক্ত করে। আমরা `return_tensors='tf'` আর্গুমেন্ট প্রদান করে টেন্সরফ্লো টেনসর চাইছি তা নির্দিষ্ট করতেও পারি:


In [4]:
tokenizer(['Hello, there'],return_tensors='tf')

{'input_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[ 101, 7592, 1010, 2045,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 5), dtype=int32, numpy=array([[1, 1, 1, 1, 1]], dtype=int32)>}

আমাদের ক্ষেত্রে, আমরা `bert-base-uncased` নামক প্রি-ট্রেইনড BERT মডেল ব্যবহার করব। *Uncased* নির্দেশ করে যে মডেলটি কেস-সংবেদনশীল নয়।

মডেলটি ট্রেইন করার সময়, আমাদের টোকেনাইজড সিকোয়েন্স ইনপুট হিসেবে প্রদান করতে হবে, এবং এজন্য আমরা ডেটা প্রসেসিং পাইপলাইন ডিজাইন করব। যেহেতু `tokenizer.encode` একটি Python ফাংশন, আমরা আগের ইউনিটে ব্যবহৃত পদ্ধতির মতোই এটি কল করার জন্য `py_function` ব্যবহার করব:


In [31]:
def process(x):
    return tokenizer.encode(x.numpy().decode('utf-8'),return_tensors='tf',padding='max_length',max_length=MAX_SEQ_LEN,truncation=True)[0]

def process_fn(x):
    s = x['title']+' '+x['description']
    e = tf.py_function(process,inp=[s],Tout=(tf.int32))
    e.set_shape(MAX_SEQ_LEN)
    return e,x['label']

এখন আমরা `BertForSequenceClassification` প্যাকেজ ব্যবহার করে প্রকৃত মডেলটি লোড করতে পারি। এটি নিশ্চিত করে যে আমাদের মডেলে ইতিমধ্যেই শ্রেণীবিভাজনের জন্য প্রয়োজনীয় আর্কিটেকচার রয়েছে, যার মধ্যে চূড়ান্ত শ্রেণীবিভাজক অন্তর্ভুক্ত। আপনি একটি সতর্কবার্তা বার্তা দেখতে পাবেন যা বলবে যে চূড়ান্ত শ্রেণীবিভাজকের ওজনগুলি প্রাথমিকভাবে নির্ধারিত নয়, এবং মডেলটি প্রাক-প্রশিক্ষণের প্রয়োজন হবে - এটি পুরোপুরি ঠিক আছে, কারণ আমরা ঠিক সেটাই করতে যাচ্ছি!


In [32]:
model = transformers.TFBertForSequenceClassification.from_pretrained(bert_model,num_labels=4,output_attentions=False)

In [33]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 109,485,316
Non-trainable params: 0
_________________________________________________________________


`summary()` থেকে আপনি দেখতে পাচ্ছেন, মডেলটিতে প্রায় ১১০ মিলিয়ন প্যারামিটার রয়েছে! অনুমান করা যায়, যদি আমরা একটি তুলনামূলকভাবে ছোট ডেটাসেটে সহজ শ্রেণীবিন্যাস কাজ করতে চাই, তবে আমরা BERT বেস লেয়ারটি প্রশিক্ষণ দিতে চাই না:


In [34]:
model.layers[0].trainable = False
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  3076      
Total params: 109,485,316
Trainable params: 3,076
Non-trainable params: 109,482,240
_________________________________________________________________


এখন আমরা প্রশিক্ষণ শুরু করতে প্রস্তুত!

> **Note**: সম্পূর্ণ স্কেলের BERT মডেল প্রশিক্ষণ করতে অনেক সময় লাগতে পারে! তাই আমরা শুধুমাত্র প্রথম ৩২টি ব্যাচের জন্য এটি প্রশিক্ষণ করব। এটি কেবল দেখানোর জন্য যে মডেল প্রশিক্ষণ কীভাবে সেট আপ করা হয়। যদি আপনি সম্পূর্ণ স্কেলের প্রশিক্ষণ চেষ্টা করতে আগ্রহী হন - তাহলে `steps_per_epoch` এবং `validation_steps` প্যারামিটারগুলি সরিয়ে ফেলুন এবং অপেক্ষা করার জন্য প্রস্তুত হন!


In [30]:
model.compile('adam','sparse_categorical_crossentropy',['acc'])
tf.get_logger().setLevel('ERROR')
model.fit(ds_train.map(process_fn).batch(32),validation_data=ds_test.map(process_fn).batch(32),steps_per_epoch=32,validation_steps=2)

32/32 [==============================] - 142s 4s/step - loss: 1.3896 - acc: 0.2500 - val_loss: 1.3863 - val_acc: 0.2480


যদি আপনি পুনরাবৃত্তির সংখ্যা বাড়ান এবং যথেষ্ট সময় অপেক্ষা করেন, এবং কয়েকটি epoch ধরে প্রশিক্ষণ চালান, তাহলে আশা করা যায় যে BERT শ্রেণীবিভাজন আমাদের সর্বোচ্চ নির্ভুলতা প্রদান করবে! এর কারণ হলো BERT ইতোমধ্যেই ভাষার গঠন বেশ ভালোভাবে বোঝে, এবং আমাদের কেবলমাত্র চূড়ান্ত শ্রেণীবিভাজকটি fine-tune করতে হয়। তবে, যেহেতু BERT একটি বড় মডেল, পুরো প্রশিক্ষণ প্রক্রিয়াটি অনেক সময় নেয় এবং উল্লেখযোগ্য কম্পিউটেশনাল ক্ষমতা প্রয়োজন হয়! (GPU, এবং সম্ভব হলে একাধিক GPU)।

> **Note:** আমাদের উদাহরণে, আমরা সবচেয়ে ছোট প্রি-ট্রেইনড BERT মডেলগুলোর একটি ব্যবহার করেছি। বড় মডেলগুলো সম্ভবত আরও ভালো ফলাফল দিতে পারে।


## মূল বিষয়বস্তু

এই ইউনিটে, আমরা **ট্রান্সফর্মারস** ভিত্তিক অত্যন্ত সাম্প্রতিক মডেল আর্কিটেকচারগুলো দেখেছি। আমরা এগুলো আমাদের টেক্সট শ্রেণীবিন্যাস কাজের জন্য প্রয়োগ করেছি, তবে একইভাবে, BERT মডেলগুলোকে সত্তা নির্ণয়, প্রশ্নোত্তর এবং অন্যান্য NLP কাজের জন্যও ব্যবহার করা যেতে পারে।

ট্রান্সফর্মার মডেলগুলো NLP-তে বর্তমানের সর্বাধুনিক অবস্থান উপস্থাপন করে, এবং বেশিরভাগ ক্ষেত্রেই এটি এমন একটি সমাধান হওয়া উচিত যার মাধ্যমে আপনি কাস্টম NLP সমাধান বাস্তবায়নের সময় পরীক্ষা-নিরীক্ষা শুরু করবেন। তবে, উন্নত নিউরাল মডেল তৈরি করতে চাইলে এই মডিউলে আলোচনা করা পুনরাবৃত্ত নিউরাল নেটওয়ার্কের মৌলিক নীতিগুলো বোঝা অত্যন্ত গুরুত্বপূর্ণ।



---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদ প্রদানের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা তার জন্য দায়ী থাকব না।
